In [21]:
import pandas as pd

from twelvelabs import TwelveLabs
from twelvelabs.types import VideoSegment
from twelvelabs.embed import TasksStatusResponse

from twelvelabs.indexes import IndexesCreateRequestModelsItem
from twelvelabs.tasks import TasksRetrieveResponse

In [22]:
TL_API_KEY = "tlk_3R75W163CAEGN82XBJB8P1VYS6XB"

In [23]:
client = TwelveLabs(api_key=TL_API_KEY)

In [24]:
index = client.indexes.retrieve(index_id="68e16c4864ff05606e152396")

In [44]:
import requests

url = "https://api.twelvelabs.io/v1.3/tasks"

files = { "video_file": open('goal2-gk0xusut_YK88uF39.mp4', 'rb') }

In [45]:
payload = {
    "index_id": "68e16c4864ff05606e152396",
}
headers = {"x-api-key": TL_API_KEY}

In [46]:
response = requests.post(url, data=payload, files=files, headers=headers)

print(response.json())

{'_id': '68e17b1a17b39f617835d2d4', 'video_id': '68e17b1a17b39f617835d2d4'}


In [47]:
result = client.gist(
    video_id="68e17b1a17b39f617835d2d4",
    types=["title", "topic", "hashtag"]
)

In [48]:
print("Result ID:", result.id)

if result.title is not None:
    print("Title:", result.title)

if result.topics is not None:
    print("Topics:")
    for topic in result.topics:
        print(f"  - {topic}")

if result.hashtags is not None:
    print("Hashtags:")
    for hashtag in result.hashtags:
        print(f"  - {hashtag}")

if result.usage is not None:
    print(f"Output tokens: {result.usage.output_tokens}")


Result ID: f29a13b2-e1cc-43c3-bc40-a8d990cb05e0
Title: Number 7's Goal: A Celebratory Moment on the Soccer Field
Topics:
  - Soccer Game Highlights
Hashtags:
  - Soccer
  - Goal
  - Teamwork
  - Celebration
  - Uniforms
Output tokens: 30


In [50]:
from twelvelabs import TwelveLabs

response = client.analyze_stream(
    video_id="68e17b1a17b39f617835d2d4",
    prompt="Break down the video by main event and timestamp",
    temperature=0.2
)

for chunk in response:
    if hasattr(chunk, 'event_type'):
        if chunk.event_type == "stream_start":
            print("Stream started")
        elif chunk.event_type == "text_generation":
            print(chunk.text, end="")
        elif chunk.event_type == "stream_end":
            print("\nStream ended")
            if chunk.metadata:
                print(f"Metadata: {chunk.metadata}")


Stream started
The video can be broken down into two main events:

1. **Goal Scoring Event (0s (00:00) - 3s (00:03)):** Early in the video, a soccer player in a blue uniform takes a shot at the goal. The ball travels towards the net, and despite the goalkeeper's attempt to block it, the ball hits the back of the net, indicating a successful goal.

2. **Celebration Event (3s (00:03) - 8s (00:08)):** Immediately after scoring, the player who took the shot begins to celebrate by running along the sideline, raising his arms in triumph. The camera follows him closely, capturing his excitement. The crowd in the background can be seen cheering, adding to the celebratory atmosphere.
Stream ended
Metadata: generation_id='96fd28d8-b18d-4618-9fb3-6814c39e866c' usage=TokenUsage(output_tokens=170)
